#APIs test

In [2]:
!pip install python-telegram-bot gspread oauth2client requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 25.8 MB/s eta 0:00:00


In [4]:
import logging, requests, datetime
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters
import gspread
from oauth2client.service_account import ServiceAccountCredentials

Telegram API

In [5]:
bot_token = "enter token"
app = ApplicationBuilder().token(bot_token).build()


Apollo API

In [25]:
import requests

api_key = "xzy"  # Replace with your actual key

url = "https://api.apollo.io/v1/users/search"

headers = {
    "Cache-Control": "no-cache",
    "Content-Type": "application/json",
    "X-Api-Key": api_key
}

params = {
    "page": 1,
    "per_page": 10  # You can adjust this as needed
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

if data.get("users"):
    for user in data["users"]:
        name = f"{user.get('first_name')} {user.get('last_name')}"
        email = user.get("email")
        title = user.get("title")
        print(f"Name: {name}")
        print(f"Title: {title}")
        print(f"Email: {email}")
        print("-" * 30)
else:
    print("Error:", data.get("message", "No users found or invalid API key."))


Name: Rusheel Sharma
Title: None
Email: rusheelhere@gmail.com
------------------------------


Google sheets API

In [7]:
from google.colab import files
uploaded = files.upload()


Saving internship-cste-apollo-647f2fceba72.json to internship-cste-apollo-647f2fceba72.json


In [8]:
!pip install gspread oauth2client


In [13]:

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/internship-cste-apollo-647f2fceba72.json", scope)
client = gspread.authorize(creds)

sheet = client.open("Apollo search logs").sheet1

# append dummy data
sheet.append_row(["2025-05-10 12:00", "Find details of Sundar Pichai", "Sundar Pichai", "CEO", "Google", "example@google.com", "https://linkedin.com/in/sundarpichai"])
print("✅ Data written to sheet successfully.")


✅ Data written to sheet successfully.


In [2]:
import requests
import time
import gspread
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials

# --- CONFIG ---
TELEGRAM_TOKEN = "xyz"
APOLLO_API_KEY = "xyz"
#replace with actual keys
GOOGLE_SHEETS_JSON = "/content/internship-cste-apollo-647f2fceba72.json"
SHEET_NAME = "Apollo search logs"

# --- SETUP GOOGLE SHEETS ---
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_SHEETS_JSON, scope)
client = gspread.authorize(creds)
sheet = client.open(SHEET_NAME).sheet1

# --- HELPER FUNCTIONS ---

def extract_name(msg):
    msg = msg.lower()
    if "find details of" in msg:
        return msg.split("find details of")[-1].strip()
    elif "email of" in msg:
        return msg.split("email of")[-1].strip()
    return msg.strip()

def search_apollo(name):
    url = "https://api.apollo.io/v1/users/search"
    headers = {
        "Cache-Control": "no-cache",
        "Content-Type": "application/json",
        "X-Api-Key": APOLLO_API_KEY
    }
    params = {
        "q_person_name": name,
        "page": 1,
        "per_page": 1,
        "enrichment_status": "enriched"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return None, f"API Error: {response.status_code} - {response.text}"

    data = response.json()
    if not data.get("people"):
        return None, "No results found."

    person = data["people"][0]
    result = {
        "Name": person.get("name"),
        "Title": person.get("title"),
        "Company": person.get("organization", {}).get("name"),
        "Email": person.get("email"),
        "LinkedIn": person.get("linkedin_url")
    }
    return result, None

def log_to_sheet(prompt, result):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row = [now, prompt] + list(result.values())
    sheet.append_row(row)

def format_result(info):
    return "\n".join([f"{k}: {v}" for k, v in info.items() if v])

def send_message(chat_id, text):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {"chat_id": chat_id, "text": text}
    requests.post(url, json=payload)

# --- POLLING TELEGRAM ---

print("Bot is running... Press Stop to quit.")

last_update_id = None

while True:
    try:
        updates_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/getUpdates"
        if last_update_id:
            updates_url += f"?offset={last_update_id + 1}"

        response = requests.get(updates_url)
        data = response.json()

        for update in data.get("result", []):
            last_update_id = update["update_id"]
            message = update.get("message", {})
            chat_id = message["chat"]["id"]
            text = message.get("text", "")

            if not text:
                continue

            print(f"Received: {text}")
            name = extract_name(text)
            result, error = search_apollo(name)

            if error:
                send_message(chat_id, f"❌ {error}")
            else:
                formatted = format_result(result)
                send_message(chat_id, f"✅ Found:\n{formatted}")
                log_to_sheet(text, result)

        time.sleep(5)

    except KeyboardInterrupt:
        print("Bot stopped.")
        break
    except Exception as e:
        print("Error:", str(e))
        time.sleep(5)


Bot is running... Press Stop to quit.
Bot stopped.
